# Решение команды Provinciya для кейса "В продакшен - зарплата", хакатона ИИ Омск. Раздел: Предсказание вакансии

## Вступительная база

### Установка модулей

### Подключение библиотек

In [89]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

### Настройки отображения Pandas

In [2]:
pd.set_option('display.max_columns', 1500)
pd.set_option('display.max_rows', 1500)

## Подготовка датасета

### Загрузка

In [5]:
df_1 = pd.read_csv('TRAIN_RES_1.csv', encoding = 'UTF-8')

In [6]:
df_2 = pd.read_csv('TRAIN_RES_2.csv', encoding = 'UTF-8')

In [24]:
df_3 = pd.read_csv('TRAIN_RES_3.csv', encoding = 'UTF-8')

In [25]:
df_4 = pd.read_csv('TRAIN_RES_4.csv', encoding = 'UTF-8')

In [26]:
df_5 = pd.read_csv('TRAIN_RES_4.csv', encoding = 'UTF-8')

In [30]:
df = pd.concat([df_1, df_2, df_3, df_4, df_5], axis = 0)

### Дропнем ненужные строки и получим сэмпл, с которым будем работать далее

In [33]:
df.drop_duplicates(inplace = True)

In [35]:
df.shape

(5697482, 6)

In [37]:
df.head()

,achievements,achievements_modified,company_name,demands,id_cv,job_title
0,NaN,NaN,Управление социальной защиты населения Сердобс...,"Работала,непосредственно, с федеральными льгот...",52860148-26da-11e8-a184-9122a281f90e,инспектор
1,NaN,NaN,Администрация города Сердобска,Работа с реестром муниципальной собственности,52860148-26da-11e8-a184-9122a281f90e,инспектор
2,"Работы не закончены, т.к. нет настоящего желан...","Работы не закончены, т.к. нет настоящего желан...",январь 2016-настоящее времяООО «АРТЕЛЬ СТАРАТЕ...,Курирую работы по рудному направлению: соавтор...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог
3,NaN,NaN,"ОАО «Васильевский рудник», п.Мотыгино, Красноя...",ГРП. Документация керна разведочных скважин,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог
4,NaN,NaN,"ООО «Артель старателей «ТАЛ»», Усть-Нера, Респ...",Поисково-разведочные работы на месторождении р...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог


In [36]:
df['job_title'].value_counts()

специалист                                            524861
менеджер                                              490578
бухгалтер                                             311718
инженер                                               245139
оператор                                              221655
                                                       ...  
музыковед                                                  1
рамповщик                                                  1
подручный вальцовщика стана холодного проката труб         1
выемщик варочных камер                                     1
брейдер                                                    1
Name: job_title, Length: 1903, dtype: int64

In [32]:
df['job_title'].nunique()

1903

In [38]:
df['job_title'].shape

(5697482,)

Возможно, стоит убрать часть ниже

In [40]:
grouped = df.groupby('job_title')

# Шаг 2: Обработка строк с одинаковым id_cv внутри каждой группы
def process_group(group):
    # Группировка по id_cv
    cv_grouped = group.groupby('id_cv')
    
    # Для каждой группы с одинаковым id_cv
    result = []
    for _, cv_group in cv_grouped:
        if len(cv_group) > 1:
            # Удаляем строки с пропусками в achievements_modified или demands
            non_missing = cv_group.dropna(subset=['achievements_modified', 'demands'])
            if not non_missing.empty:
                result.append(non_missing)
            else:
                # Если все строки с пропусками, сохраняем оригинальные
                result.append(cv_group)
        else:
            result.append(cv_group)
    
    # Объединяем все выбранные строки в одну таблицу
    return pd.concat(result)

In [41]:
sampled_df = grouped.apply(process_group).reset_index(drop=True)

In [42]:
sampled_df['job_title'].nunique()

1903

In [43]:
sampled_df['job_title'].value_counts()

специалист                503361
менеджер                  469071
бухгалтер                 294715
инженер                   232610
оператор                  217841
                           ...  
клишист                        1
ковровщик                      1
верховой доменной печи         1
пробист плоской печати         1
кооператор                     1
Name: job_title, Length: 1903, dtype: int64

In [44]:
sampled_df.shape

(5531858, 6)

In [49]:
sampled_df.to_csv('sampled_jobs.csv', index=False)

### Аггрегируем собранную информацию

In [50]:
df = pd.read_csv('sampled_jobs.csv', encoding = 'UTF-8')

In [53]:
df.head()

,achievements,achievements_modified,company_name,demands,id_cv,job_title
0,NaN,NaN,"ООО ""АварКом""",Выезд на место ДТП. Составление схемы ДТП. Опр...,01235ff0-847f-11ea-a5e5-69632329477c,аварийный комиссар
1,NaN,NaN,ООО “Сибирская аварийная служба 54”,- Контроль аварийного комиссара при выезде на ...,02101d10-88b5-11ea-93b5-239861b1f67b,аварийный комиссар
2,NaN,NaN,ООО “Сибирская аварийная служба 54”,- Контроль аварийного комиссара при выезде на ...,02101d10-88b5-11ea-93b5-239861b1f67b,аварийный комиссар
3,NaN,NaN,"ООО""Автопартнёр Бузулук""","1. Выезд на место дтп, оформление участников дтп",0506c8e0-2ac7-11e9-9058-037acc02728d,аварийный комиссар
4,NaN,NaN,"ООО ""Крепость""",Оформление ДТП,070946c0-8450-11ea-a79e-bfd13399602c,аварийный комиссар


In [55]:
df['user_info'] = df.apply(lambda row: 
    (str(row['achievements_modified']) if pd.notnull(row['achievements_modified']) else str(row['achievements']) if pd.notnull(row['achievements']) else '') +
    (', ' + str(row['company_name']) if pd.notnull(row['company_name']) else '') +
    (', ' + str(row['demands']) if pd.notnull(row['demands']) else ''),
    axis=1
)

In [57]:
df.head(10)

,achievements,achievements_modified,company_name,demands,id_cv,job_title,user_info
0,NaN,NaN,"ООО ""АварКом""",Выезд на место ДТП. Составление схемы ДТП. Опр...,01235ff0-847f-11ea-a5e5-69632329477c,аварийный комиссар,", ООО ""АварКом"", Выезд на место ДТП. Составлен..."
1,NaN,NaN,ООО “Сибирская аварийная служба 54”,- Контроль аварийного комиссара при выезде на ...,02101d10-88b5-11ea-93b5-239861b1f67b,аварийный комиссар,", ООО “Сибирская аварийная служба 54”, - Контр..."
2,NaN,NaN,ООО “Сибирская аварийная служба 54”,- Контроль аварийного комиссара при выезде на ...,02101d10-88b5-11ea-93b5-239861b1f67b,аварийный комиссар,", ООО “Сибирская аварийная служба 54”, - Контр..."
3,NaN,NaN,"ООО""Автопартнёр Бузулук""","1. Выезд на место дтп, оформление участников дтп",0506c8e0-2ac7-11e9-9058-037acc02728d,аварийный комиссар,", ООО""Автопартнёр Бузулук"", 1. Выезд на место ..."
4,NaN,NaN,"ООО ""Крепость""",Оформление ДТП,070946c0-8450-11ea-a79e-bfd13399602c,аварийный комиссар,", ООО ""Крепость"", Оформление ДТП"
5,нет,нет,ООО «Оргтранс-Авто»,"выезд на место дтп, составление схемы дтп, фот...",09131100-beb3-11eb-ae36-ab5d2eb93a75,аварийный комиссар,"нет, ООО «Оргтранс-Авто», выезд на место дтп, ..."
6,Открытие филиалов по республике Татарстан и п-...,Открытие филиалов по республике Татарстан и п-...,Региональная Служба Аваркомов,"Оформление дтп, обращение в страховую компанию...",091869a0-7bea-11ea-a1d8-b905beff6f7a,аварийный комиссар,Открытие филиалов по республике Татарстан и п-...
7,NaN,NaN,"ООО ""Юравтоцентр""",Оказание услуг по оформлению дорожного транспо...,0ac09450-9f29-11ea-b210-b905beff6f7a,аварийный комиссар,", ООО ""Юравтоцентр"", Оказание услуг по оформле..."
8,NaN,NaN,ООО «Аварком»,"Оформление ДТП, выдача справок с ГИБДД",0fec93b0-cc2c-11e8-9c43-e37b4be0b9ed,аварийный комиссар,", ООО «Аварком», Оформление ДТП, выдача справо..."
9,Успешно справлялся с поставленными планами по ...,Успешно справлялся с поставленными планами по ...,"Архангельское бюро оценки, ООО","Продажа юридических услуг участникам ДТП, конс...",16308170-c431-11e9-9b9a-736ab11edb0c,аварийный комиссар,Успешно справлялся с поставленными планами по ...


In [59]:
df[['id_cv', 'job_title', 'user_info']].to_csv('jobs_data.csv', index=False)

In [62]:
df = pd.read_csv('jobs_data.csv')

## Предсказание топ 5 подходящих вакансий с помощью трансформера

### Синтетические данные для теста

In [69]:
df = df.sample(10)

In [70]:
df

,id_cv,job_title,user_info
4991509,664b8370-2a2a-11eb-87f0-bfd13399602c,уборщик,", Государственное бюджетное учреждение здравоо..."
2602721,8d2a5f50-1ae8-11e9-95e0-ef76bd2a03c1,менеджер,", ООО Камышинский завод слесарно-монтажного ин..."
1705935,fdfb73c0-03b9-11eb-9b0b-69632329477c,инспектор,", Внуковская таможня, защита экономических инт..."
1260048,27914f90-be91-11e8-bfef-736ab11edb0c,заместитель,", ФГАОУ ВО ""КФУ им. В.И. Вернадского"", Планиро..."
1926751,f6d085e0-d00b-11ea-87bd-ef76bd2a03c1,комплектовщик,", ОАО Пермский мясокомбинат, Комплектация заяв..."
1613050,f7d34850-4c13-11ea-8c26-037acc02728d,инженер,", ООО ""МПЗ"", Москва, Договорная и рекламационн..."
864430,0c498cb0-80c9-11ea-b710-736ab11edb0c,гардеробщик,", МЛПУ ИРБ стомат.пол-ки, Приём одежды"
1791493,7b8f714a-0662-11e8-8956-037acc02728d,кассир,", ООО ""Салют торг"", работа на кассе, выкладка ..."
3205779,5128e480-d5ed-11e8-a08a-ef76bd2a03c1,отделочник,", АО ""Владимирская обувная фабрика"", Полировка..."
1285647,5761dfb0-1f32-11eb-b6f7-e37b4be0b9ed,заместитель,В период с 2016 по 2019 год участие в работе к...


### Получение эмбеддингов

Проверить выбор эмбеддинга

In [ ]:
model_name = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [66]:
def get_embeddings(texts):
    # Токенизация текстов с добавлением специальных токенов и выравниванием
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    
    # Прогон через модель для получения эмбеддингов
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Используем среднее по всем токенам для получения одного эмбеддинга на текст
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

In [ ]:
#df_vac = pd.read_csv('')

In [78]:
candidate_embeddings = get_embeddings(list(df['user_info'].values))

In [83]:
job_embeddings = get_embeddings(list(df_vac['vac_info'].values))

In [91]:
vacancies = list(df_vac['vacancies'].values)

### Ищем топ 5 вакансий

In [87]:
similarity_matrix = cosine_similarity(candidate_embeddings, job_embeddings)

In [92]:
# Создание итогового списка для каждого кандидата
top_vacancies_per_candidate = []

for i, similarities in enumerate(similarity_matrix):
    # Индексы топ-5 вакансий
    top_indices = np.argsort(similarities)[-5:][::-1]
    top_vacancies = [(vacancies[j], similarities[j]) for j in top_indices]
    top_vacancies_per_candidate.append(top_vacancies)

Кандидат 1:
Вакансия: заместитель, Сходство: 0.5077
Вакансия: инспектор, Сходство: 0.3606
Вакансия: гардеробщик, Сходство: 0.3456
Вакансия: отделочник, Сходство: 0.3259
Вакансия: кассир, Сходство: 0.3254


Кандидат 2:
Вакансия: заместитель, Сходство: 0.6200
Вакансия: комплектовщик, Сходство: 0.5114
Вакансия: инспектор, Сходство: 0.5070
Вакансия: кассир, Сходство: 0.4982
Вакансия: отделочник, Сходство: 0.4815


Кандидат 3:
Вакансия: заместитель, Сходство: 0.4101
Вакансия: инспектор, Сходство: 0.3648
Вакансия: гардеробщик, Сходство: 0.3515
Вакансия: инженер, Сходство: 0.3488
Вакансия: отделочник, Сходство: 0.3345


Кандидат 4:
Вакансия: заместитель, Сходство: 0.5396
Вакансия: кассир, Сходство: 0.4810
Вакансия: инспектор, Сходство: 0.4451
Вакансия: гардеробщик, Сходство: 0.4224
Вакансия: инженер, Сходство: 0.4129


Кандидат 5:
Вакансия: заместитель, Сходство: 0.4488
Вакансия: отделочник, Сходство: 0.4262
Вакансия: комплектовщик, Сходство: 0.4243
Вакансия: гардеробщик, Сходство: 0.3856
Вак

In [95]:
# Вывод результатов
for i, top_vacancies in enumerate(top_vacancies_per_candidate):
    print(f"Кандидат {i+1}:")
    for vacancy, similarity in top_vacancies:
        print(f"Вакансия: {vacancy}, Сходство: {similarity:.4f}")
    print("\n")

Кандидат 1:
Вакансия: заместитель, Сходство: 0.5077
Вакансия: инспектор, Сходство: 0.3606
Вакансия: гардеробщик, Сходство: 0.3456
Вакансия: отделочник, Сходство: 0.3259
Вакансия: кассир, Сходство: 0.3254


Кандидат 2:
Вакансия: заместитель, Сходство: 0.6200
Вакансия: комплектовщик, Сходство: 0.5114
Вакансия: инспектор, Сходство: 0.5070
Вакансия: кассир, Сходство: 0.4982
Вакансия: отделочник, Сходство: 0.4815


Кандидат 3:
Вакансия: заместитель, Сходство: 0.4101
Вакансия: инспектор, Сходство: 0.3648
Вакансия: гардеробщик, Сходство: 0.3515
Вакансия: инженер, Сходство: 0.3488
Вакансия: отделочник, Сходство: 0.3345


Кандидат 4:
Вакансия: заместитель, Сходство: 0.5396
Вакансия: кассир, Сходство: 0.4810
Вакансия: инспектор, Сходство: 0.4451
Вакансия: гардеробщик, Сходство: 0.4224
Вакансия: инженер, Сходство: 0.4129


Кандидат 5:
Вакансия: заместитель, Сходство: 0.4488
Вакансия: отделочник, Сходство: 0.4262
Вакансия: комплектовщик, Сходство: 0.4243
Вакансия: гардеробщик, Сходство: 0.3856
Вак

In [94]:
df.head(10)

,id_cv,job_title,user_info
4991509,664b8370-2a2a-11eb-87f0-bfd13399602c,уборщик,", Государственное бюджетное учреждение здравоо..."
2602721,8d2a5f50-1ae8-11e9-95e0-ef76bd2a03c1,менеджер,", ООО Камышинский завод слесарно-монтажного ин..."
1705935,fdfb73c0-03b9-11eb-9b0b-69632329477c,инспектор,", Внуковская таможня, защита экономических инт..."
1260048,27914f90-be91-11e8-bfef-736ab11edb0c,заместитель,", ФГАОУ ВО ""КФУ им. В.И. Вернадского"", Планиро..."
1926751,f6d085e0-d00b-11ea-87bd-ef76bd2a03c1,комплектовщик,", ОАО Пермский мясокомбинат, Комплектация заяв..."
1613050,f7d34850-4c13-11ea-8c26-037acc02728d,инженер,", ООО ""МПЗ"", Москва, Договорная и рекламационн..."
864430,0c498cb0-80c9-11ea-b710-736ab11edb0c,гардеробщик,", МЛПУ ИРБ стомат.пол-ки, Приём одежды"
1791493,7b8f714a-0662-11e8-8956-037acc02728d,кассир,", ООО ""Салют торг"", работа на кассе, выкладка ..."
3205779,5128e480-d5ed-11e8-a08a-ef76bd2a03c1,отделочник,", АО ""Владимирская обувная фабрика"", Полировка..."
1285647,5761dfb0-1f32-11eb-b6f7-e37b4be0b9ed,заместитель,В период с 2016 по 2019 год участие в работе к...


## Предсказание топ 5 подходящих вакансий с помощью ключевых слов